## data prep

In [670]:
import pandas

In [671]:
train=pandas.read_csv("train.csv")

In [672]:
train.shape

(891, 12)

In [673]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [674]:
dv='Survived'

In [710]:
train.dtypes

PassengerId         int64
Survived            int64
Pclass              int64
Name               object
Sex                object
Age               float64
SibSp               int64
Parch               int64
Ticket             object
Fare              float64
Cabin              object
Embarked         category
cabinflag           int64
deck               object
orticket            int64
lastname           object
title              object
family              int64
fareperperson     float64
dtype: object

In [676]:
train.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [677]:
nrow=train.PassengerId.count()

In [678]:
#nulls=train.isnull().apply(lambda x: sum(x)/nrow)

In [679]:
#nulls=nulls.index[nulls>0.5].tolist()

In [680]:
#train2=train.drop(nulls,1)

In [681]:
#from sklearn.preprocessing import Imputer

In [682]:
#imp=Imputer(strategy='most_frequent',axis=1,copy=True)

In [683]:
#imp.transform(train2)

# variables creation

In [684]:
train=train.assign(cabinflag=train.Cabin.notnull().astype('int64'))

In [685]:
train=train.assign(deck=train.Cabin.astype('str').apply(lambda x: x[0]))

In [686]:
train=train.assign(orticket=train.Ticket.apply(str.isnumeric).astype('int64'))

In [687]:
train=train.assign(lastname=train.Name.apply(lambda x: x.split(",",1)[0]))

In [688]:
train=train.assign(title=train.Name.apply(lambda x: x.split(",",1)[1].split(".",1)[0].strip()))

In [689]:
train=train.assign(family=train.SibSp+train.Parch)

In [690]:
train=train.assign(fareperperson=train.Fare/(train.family+1))

In [691]:
train.loc[train.Embarked.isnull(),'Embarked']=pandas.Series(['U'],index=train[train.Embarked.isnull()].index)

In [667]:
#fare as % of median fare for class-deck

In [668]:
# travelling with spouse, parents, children or alone

In [716]:
#types of tickets

## exploration

In [669]:
train2.title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [693]:
train.groupby(by='title').apply(lambda x: x.shape[0])

title
Capt              1
Col               2
Don               1
Dr                7
Jonkheer          1
Lady              1
Major             2
Master           40
Miss            182
Mlle              2
Mme               1
Mr              517
Mrs             125
Ms                1
Rev               6
Sir               1
the Countess      1
dtype: int64

In [694]:
train.groupby('Ticket').apply(lambda x: x.shape[0]).order(ascending=False).head()

Ticket
CA. 2343    7
1601        7
347082      7
347088      6
CA 2144     6
dtype: int64

In [695]:
train.groupby('orticket').apply(lambda x: x.shape[0]).order(ascending=False).head()

orticket
1    661
0    230
dtype: int64

In [696]:
train.groupby(['orticket','Pclass']).apply(lambda x: x.shape[0]).order(ascending=False).head()

orticket  Pclass
1         3         382
          1         152
          2         127
0         3         109
          1          64
dtype: int64

In [697]:
train.groupby(by='lastname').apply(lambda x: x.shape[0]).order(ascending=False).head()

lastname
Andersson    9
Sage         7
Skoog        6
Johnson      6
Panula       6
dtype: int64

In [698]:
train.loc[train.family.order(ascending=False).index,'lastname'].unique()[0:5]

array(['Sage', 'Goodwin', 'Andersson', 'Asplund', 'Panula'], dtype=object)

In [699]:
train.groupby('Pclass').Cabin.count()

Pclass
1    176
2     16
3     12
Name: Cabin, dtype: int64

In [700]:
train.groupby('Pclass').Cabin.unique()

Pclass
1    [C85, C123, E46, C103, A6, C23 C25 C27, nan, B...
2                [nan, D56, F33, E101, F2, F4, D, E77]
3       [nan, G6, F G73, F E69, E10, F G63, E121, F38]
Name: Cabin, dtype: object

In [701]:
train.groupby('title').Cabin.count()

title
Capt             1
Col              1
Don              0
Dr               3
Jonkheer         0
Lady             1
Major            2
Master           7
Miss            47
Mlle             2
Mme              1
Mr              93
Mrs             44
Ms               0
Rev              0
Sir              1
the Countess     1
Name: Cabin, dtype: int64

# Imputation

#### impute age with title wise median age

In [702]:
Ageimp=train.groupby('title').Age.median()

In [703]:
train.loc[train.Age.isnull(),'Age']=train.title.apply(lambda x: Ageimp[x])[train[train.Age.isnull()].index]

In [704]:
train.isnull().apply(lambda x: sum(x)/nrow)

PassengerId      0.000000
Survived         0.000000
Pclass           0.000000
Name             0.000000
Sex              0.000000
Age              0.000000
SibSp            0.000000
Parch            0.000000
Ticket           0.000000
Fare             0.000000
Cabin            0.771044
Embarked         0.000000
cabinflag        0.000000
deck             0.000000
orticket         0.000000
lastname         0.000000
title            0.000000
family           0.000000
fareperperson    0.000000
dtype: float64

In [728]:
train=train.assign(Sex=train.Sex.astype('category').cat.codes)

In [729]:
train=train.assign(Embarked=train.Embarked.astype('category').cat.codes)

In [730]:
train=train.assign(deck=train.deck.astype('category').cat.codes)

In [731]:
train=train.assign(title=train.title.astype('category').cat.codes)

# Modeling

In [733]:
dev=train.loc[:,['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','cabinflag','deck','orticket','title','family','fareperperson']]

In [734]:
dev.shape

(891, 13)

In [735]:
from sklearn.ensemble import RandomForestClassifier

In [785]:
modobj=RandomForestClassifier(random_state=111,verbose=0)

In [737]:
from sklearn.cross_validation import StratifiedKFold, StratifiedShuffleSplit

In [780]:
sss=StratifiedShuffleSplit(train.Survived.values, 5, test_size=0.2, random_state=121)

In [739]:
from sklearn.grid_search import GridSearchCV, ParameterGrid

In [786]:
GSobj=GridSearchCV(modobj,{'min_samples_leaf':[25,50],'max_depth':[3,5,7,9],'max_features':['sqrt'],'criterion':["gini"],'n_estimators':[100,200,400]},
                   scoring='accuracy',cv=sss,verbose=10000,n_jobs=-1)

In [783]:
GSobj.fit(dev,train.Survived.values)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] n_estimators=100, criterion=gini, min_samples_leaf=25, max_features=sqrt, max_depth=3 
[CV]  n_estimators=100, criterion=gini, min_samples_leaf=25, max_features=sqrt, max_depth=3, score=0.793296 -   0.2s
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[CV] n_estimators=100, criterion=gini, min_samples_leaf=25, max_features=sqrt, max_depth=3 
[CV]  n_estimators=100, criterion=gini, min_samples_leaf=25, max_features=sqrt, max_depth=3, score=0.804469 -   0.3s
[Parallel(n_jobs=1)]: Done   2 jobs       | elapsed:    0.6s
[CV] n_estimators=100, criterion=gini, min_samples_leaf=25, max_features=sqrt, max_depth=3 
[CV]  n_estimators=100, criterion=gini, min_samples_leaf=25, max_features=sqrt, max_depth=3, score=0.804469 -   0.2s
[Parallel(n_jobs=1)]: Done   3 jobs       | elapsed:    0.9s
[CV] n_estimators=100, criterion=gini, min_samples_leaf=25, max_features=sqrt, max_depth=3 
[CV]  n_estimators=100, criterion=gi

GridSearchCV(cv=StratifiedShuffleSplit(labels=[0 1 ..., 1 0], n_iter=5, test_size=0.2, random_state=121),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=111, verbose=0, warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 400], 'criterion': ['gini'], 'min_samples_leaf': [25, 50], 'max_features': ['sqrt'], 'max_depth': [3, 5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=120)

In [784]:
GSobj.best_score_

0.81005586592178769